### 获得一个 mmrotate 模型的参数量

In [9]:
import os
import mim
import torch

from mmengine.config import Config
from mmengine.registry import MODELS
from mmrotate.utils import register_all_modules
from mmdet.utils import register_all_modules as register_all_modules_mmdet


def monkey_patch_of_collections_typehint_for_mmrotate1x():
    import collections
    from collections.abc import Mapping, Sequence, Iterable
    collections.Mapping = Mapping
    collections.Sequence = Sequence
    collections.Iterable = Iterable

def get_num_parameters(module):
    """Modified from print_trainable_parameters of peft"""
    def _get_parameter_numel(param):
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            # if using DS Zero 3 and the weights are initialized empty
            num_params = param.ds_numel
        return num_params
    
    if isinstance(module, torch.Tensor):  # nn.Parameter()
        num_params = _get_parameter_numel(module)
        return num_params if module.requires_grad else 0, num_params
        
    trainable_params = 0
    all_param = 0
    for param in module.parameters():
        num_params = _get_parameter_numel(param)
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    return trainable_params, all_param

monkey_patch_of_collections_typehint_for_mmrotate1x()

register_all_modules_mmdet(init_default_scope=False)
register_all_modules(init_default_scope=True)

def print_mmrotate_model_num_params(model_name):
    save_root = f"./eval_mmrotate/{model_name}"
    mim.download('mmrotate', [model_name], dest_root=os.path.join(save_root, model_name))[0]
    cfg_fpath = os.path.join(save_root, model_name, f'{model_name}.py')
    cfg = Config.fromfile(cfg_fpath)
    model = MODELS.build(cfg.model)

    num_params = get_num_parameters(model)[1]
    print(f"{model_name} #params: {num_params:,d}")

print_mmrotate_model_num_params(model_name='rotated-fcos-le90_r50_fpn_1x_dota')
print_mmrotate_model_num_params(model_name='rotated-retinanet-rbox-le90_r50_fpn_1x_dota')

processing rotated-fcos-le90_r50_fpn_1x_dota...
rotated_fcos_r50_fpn_1x_dota_le90-d87568ed.pth exists in /mnt/petrelfs/liqingyun/florence-dota/scripts_py/eval_rfcos/rotated-fcos-le90_r50_fpn_1x_dota
Successfully dumped rotated-fcos-le90_r50_fpn_1x_dota.py to /mnt/petrelfs/liqingyun/florence-dota/scripts_py/eval_rfcos/rotated-fcos-le90_r50_fpn_1x_dota
rotated-fcos-le90_r50_fpn_1x_dota #params: 32,147,803
processing rotated-retinanet-rbox-le90_r50_fpn_1x_dota...
rotated_retinanet_obb_r50_fpn_1x_dota_le90-c0097bc4.pth exists in /mnt/petrelfs/liqingyun/florence-dota/scripts_py/eval_mmrotate/rotated-retinanet-rbox-le90_r50_fpn_1x_dota/rotated-retinanet-rbox-le90_r50_fpn_1x_dota
Successfully dumped rotated-retinanet-rbox-le90_r50_fpn_1x_dota.py to /mnt/petrelfs/liqingyun/florence-dota/scripts_py/eval_mmrotate/rotated-retinanet-rbox-le90_r50_fpn_1x_dota/rotated-retinanet-rbox-le90_r50_fpn_1x_dota
rotated-retinanet-rbox-le90_r50_fpn_1x_dota #params: 36,641,012


(None,)

### 获得一个 huggingface transformers 模型额参数量

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoModel

def get_num_parameters(module):
    """Modified from print_trainable_parameters of peft"""
    def _get_parameter_numel(param):
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            # if using DS Zero 3 and the weights are initialized empty
            num_params = param.ds_numel
        return num_params
    
    if isinstance(module, torch.Tensor):  # nn.Parameter()
        num_params = _get_parameter_numel(module)
        return num_params if module.requires_grad else 0, num_params
        
    trainable_params = 0
    all_param = 0
    for param in module.parameters():
        num_params = _get_parameter_numel(param)
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    return trainable_params, all_param

def print_transformer_model_num_params(model_name):
    if "Florence" in model_name:
        model = AutoModelForCausalLM.from_pretrained(model_name, attn_implementation="eager", trust_remote_code=True)
    elif "deepseek" in model_name:
        from lmmrotate.models.deepseek_vl2 import DeepseekVLV2ForCausalLM
        model = DeepseekVLV2ForCausalLM.from_pretrained(model_name, attn_implementation="eager", trust_remote_code=True)
    elif "llava" in model_name:
        from lmmrotate.models.llava import load_pretrained_model
        model = load_pretrained_model(model_path=model_name, model_name="llava_qwen")[1]
    else:
        model = AutoModel.from_pretrained(model_name, attn_implementation="eager", trust_remote_code=True)
    num_params = get_num_parameters(model)[1]
    print(f"{model_name} #params: {num_params:,d}")

print_transformer_model_num_params(model_name='microsoft/Florence-2-base')  # huggingface-cli download microsoft/Florence-2-base --repo-type model
print_transformer_model_num_params(model_name='microsoft/Florence-2-large')  # huggingface-cli download microsoft/Florence-2-large --repo-type model
print_transformer_model_num_params(model_name='Qwen/Qwen2-VL-2B-Instruct')  # huggingface-cli download Qwen/Qwen2-VL-2B-Instruct --repo-type model
print_transformer_model_num_params(model_name='lmms-lab/llava-onevision-qwen2-0.5b-si')  # huggingface-cli download lmms-lab/llava-onevision-qwen2-0.5b-si --repo-type model
print_transformer_model_num_params(model_name='OpenGVLab/InternVL2-1B')  # huggingface-cli download OpenGVLab/InternVL2-1B --repo-type model
print_transformer_model_num_params(model_name='OpenGVLab/InternVL2-2B')  # huggingface-cli download OpenGVLab/InternVL2-2B --repo-type model
print_transformer_model_num_params(model_name='OpenGVLab/InternVL2-8B')  # huggingface-cli download OpenGVLab/InternVL2-8B --repo-type model

# microsoft/Florence-2-base #params: 270,803,968
# microsoft/Florence-2-large #params: 828,985,344
# Qwen/Qwen2-VL-2B-Instruct #params: 1,543,714,304
# lmms-lab/llava-onevision-qwen2-0.5b-si #params: 893,359,264
# OpenGVLab/InternVL2-1B #params: 938,158,976
# OpenGVLab/InternVL2-2B #params: 2,205,754,368
# OpenGVLab/InternVL2-8B #params: 8,075,365,376